In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import math
import random
import numpy as np
import pandas as pd
from collections import Counter

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score


df= pd.read_excel('queen_por_pontuacao.xlsx', sheet_name=0)


In [ ]:
def prepare_sequences(df, drop_cols=["colocacao", "queen"]):
    # 1. ordenar por queen (id) e episódio
    df = df.sort_values(["id", "ep"]).reset_index(drop=True)

    # 2. opcional: remover coluna de colocação para não vazar
    df = df.drop(columns=[c for c in drop_cols if c in df.columns])

    # 3. codificar season (tempfranquia) se necessário - usar LabelEncoder ou one-hot
    if "tempfranquia" in df.columns:
        le_temp = LabelEncoder()
        df["tempfranquia_le"] = le_temp.fit_transform(df["tempfranquia"].astype(str))
    else:
        df["tempfranquia_le"] = 0

    # 4. agrupar por queen (id) e montar sequências
    groups = []
    for gid, g in df.groupby("id"):
        g = g.sort_values("ep")
        # features por timestep: bom, ruim, media, idade, tempfranquia_le (idade e tempfranquia constantes por queen)
        feat_cols = [c for c in ["bom", "ruim", "media", "idade", "tempfranquia_le"] if c in g.columns]
        seq = g[feat_cols].values.astype(np.float32)
        label = int(g["vencedora"].iloc[0])  # label por queen (0/1)
        queen_name = g["queen"].iloc[0] if "queen" in g.columns else str(gid)
        groups.append({
            "id": gid,
            "queen": queen_name,
            "seq": seq,
            "length": seq.shape[0],
            "label": label
        })

    return groups

In [6]:
class QueenSequenceDataset(Dataset):
    def __init__(self, groups, scaler=None):
        self.groups = groups
        # achatar todos os timesteps para treinar scaler se fornecido
        if scaler is None:
            self.scaler = StandardScaler()
            all_timesteps = np.vstack([g["seq"] for g in groups])
            self.scaler.fit(all_timesteps)
        else:
            self.scaler = scaler
        # aplicar scaler
        for g in self.groups:
            g["seq"] = self.scaler.transform(g["seq"])

    def __len__(self):
        return len(self.groups)

    def __getitem__(self, idx):
        g = self.groups[idx]
        return torch.tensor(g["seq"], dtype=torch.float32), g["length"], torch.tensor(g["label"], dtype=torch.float32)

def collate_fn(batch):
    # batch: list of (seq_tensor, length, label)
    seqs, lengths, labels = zip(*batch)
    lengths = torch.tensor(lengths, dtype=torch.long)
    labels = torch.tensor(labels, dtype=torch.float32)
    # pad sequences
    seqs_padded = nn.utils.rnn.pad_sequence(seqs, batch_first=True)  # (B, T_max, D)
    return seqs_padded, lengths, labels


In [20]:
class GRUClassifier(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=1, bidirectional=False, dropout=0.1):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.gru = nn.GRU(input_size=input_size,
                          hidden_size=hidden_size,
                          num_layers=num_layers,
                          batch_first=True,
                          bidirectional=bidirectional,
                          dropout=dropout if num_layers > 1 else 0.0)
        mult = 2 if bidirectional else 1
        self.fc = nn.Linear(hidden_size * mult, 1)

    def forward(self, x, lengths, return_seq=False):
        packed = nn.utils.rnn.pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, h_n = self.gru(packed)
        out, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)  # (B, T, H*mult)

        if return_seq:
            # saída em cada timestep
            logits = self.fc(out).squeeze(-1)  # (B, T)
            return logits
        else:
            # só último estado
            if self.bidirectional:
                last_fw = h_n[-2, :, :]
                last_bw = h_n[-1, :, :]
                h = torch.cat([last_fw, last_bw], dim=1)
            else:
                h = h_n[-1, :, :]
            logits = self.fc(h).squeeze(1)
            return logits


In [15]:
def make_episode_table(results):
    # descobrir o máximo de episódios
    max_len = max(len(r["probs"]) for r in results)
    rows = []
    for r in results:
        row = [r["queen"]]
        row += [f"{p:.2f}" for p in r["probs"]]
        # padding se tiver menos episódios
        while len(row) < max_len + 1:
            row.append("")
        rows.append(row)
    cols = ["queen"] + [f"ep{i+1}" for i in range(max_len)]
    return pd.DataFrame(rows, columns=cols)


In [14]:
def predict_episode_by_episode(model, dataset, device="cpu"):
    model.eval()
    all_results = []
    with torch.no_grad():
        for i in range(len(dataset)):
            seq, length, label = dataset[i]
            seq = seq.unsqueeze(0).to(device)   # (1, T, D)
            length = torch.tensor([length])
            logits = model(seq, length, return_seq=True)  # (1, T)
            probs = torch.sigmoid(logits).cpu().numpy().flatten()
            queen_id = dataset.groups[i]["id"]
            all_results.append({
                "id": queen_id,
                "queen": dataset.groups[i].get("queen", str(queen_id)),
                "probs": probs,
                "label": int(label.item())
            })
    return all_results


In [8]:
def train_epoch(model, loader, opt, criterion, device):
    model.train()
    total_loss = 0.0
    for seqs, lengths, labels in loader:
        seqs, lengths, labels = seqs.to(device), lengths.to(device), labels.to(device)
        opt.zero_grad()
        logits = model(seqs, lengths)
        loss = criterion(logits, labels)
        loss.backward()
        opt.step()
        total_loss += loss.item() * seqs.size(0)
    return total_loss / len(loader.dataset)

def eval_model(model, loader, device):
    model.eval()
    preds = []
    probs = []
    trues = []
    with torch.no_grad():
        for seqs, lengths, labels in loader:
            seqs, lengths = seqs.to(device), lengths.to(device)
            logits = model(seqs, lengths)
            prob = torch.sigmoid(logits).cpu().numpy()
            pred = (prob >= 0.5).astype(int)
            preds.extend(pred.tolist())
            probs.extend(prob.tolist())
            trues.extend(labels.numpy().astype(int).tolist())
    # métricas
    p, r, f1, _ = precision_recall_fscore_support(trues, preds, average="binary", zero_division=0)
    auc = None
    try:
        auc = roc_auc_score(trues, probs)
    except:
        auc = float("nan")
    return {"precision": p, "recall": r, "f1": f1, "auc": auc}

In [11]:
def run_pipeline(df,
                 test_size=0.2,
                 random_state=42,
                 batch_size=16,
                 hidden_size=64,
                 epochs=30,
                 device=None):
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    groups = prepare_sequences(df)

    # dividir em treino/val por tempfranquia (evita vazar informações entre temporadas)
    # criar lista de season por id (assumindo season é igual nas linhas de um id)
    # se não houver tempfranquia, faremos split randômico por id
    df_id2season = df.groupby("id")["tempfranquia"].first() if "tempfranquia" in df.columns else None
    ids = [g["id"] for g in groups]
    if df_id2season is not None:
        id_list = np.array(ids)
        seasons = np.array([str(df_id2season[i]) for i in id_list])
        # stratify by season could still leak winners distribution; uma alternativa simples: stratify por label
        labels = np.array([g["label"] for g in groups])
        train_idx, test_idx = train_test_split(range(len(groups)), test_size=test_size, random_state=random_state, stratify=labels)
    else:
        labels = np.array([g["label"] for g in groups])
        train_idx, test_idx = train_test_split(range(len(groups)), test_size=test_size, random_state=random_state, stratify=labels)

    train_groups = [groups[i] for i in train_idx]
    test_groups = [groups[i] for i in test_idx]

    # criar scaler a partir de treino
    dataset_train = QueenSequenceDataset(train_groups, scaler=None)
    scaler = dataset_train.scaler
    dataset_test = QueenSequenceDataset(test_groups, scaler=scaler)

    # lidar com imbalance: WeightedRandomSampler por label na dataset_train
    labels_train = np.array([g["label"] for g in train_groups])
    class_counts = Counter(labels_train.tolist())
    # se classe 1 muito rara, dar weight inverso da frequência
    class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}
    sample_weights = np.array([class_weights[int(lbl)] for lbl in labels_train])
    sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

    loader_train = DataLoader(dataset_train, batch_size=batch_size, sampler=sampler, collate_fn=collate_fn)
    loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    input_size = dataset_train[0][0].shape[1]
    model = GRUClassifier(input_size=input_size, hidden_size=hidden_size, bidirectional=True).to(device)

    # Loss com peso de classes (opcional)
    # calcular peso para classe positiva na BCEWithLogitsLoss
    pos_weight = None
    if 1 in class_counts:
        neg = class_counts.get(0, 0)
        pos = class_counts.get(1, 0)
        if pos == 0:
            pos_weight = None
        else:
            pos_weight = torch.tensor([neg / pos], dtype=torch.float32).to(device)

    if pos_weight is not None:
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    else:
        criterion = nn.BCEWithLogitsLoss()

    opt = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

    best_f1 = -1
    best_state = None

    for epoch in range(1, epochs + 1):
        loss = train_epoch(model, loader_train, opt, criterion, device)
        metrics = eval_model(model, loader_test, device)
        print(f"Epoch {epoch:02d} | Loss {loss:.4f} | val f1 {metrics['f1']:.4f} prec {metrics['precision']:.4f} rec {metrics['recall']:.4f} auc {metrics['auc']:.4f}")
        if metrics["f1"] > best_f1:
            best_f1 = metrics["f1"]
            best_state = {k:v.cpu() for k,v in model.state_dict().items()}

    # carregar melhor modelo
    if best_state is not None:
        model.load_state_dict(best_state)
    final_metrics = eval_model(model, loader_test, device)
    print("== Final metrics on test set:", final_metrics)
    return {"model": model, "scaler": scaler, "dataset_train": dataset_train, "dataset_test": dataset_test, "metrics": final_metrics}


In [25]:
def run_pipeline2(df,
                 test_size=0.2,
                 random_state=42,
                 batch_size=16,
                 hidden_size=64,
                 epochs=30,
                 device=None):
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    groups = prepare_sequences(df)

    # dividir em treino/val por tempfranquia (evita vazar informações entre temporadas)
    # criar lista de season por id (assumindo season é igual nas linhas de um id)
    # se não houver tempfranquia, faremos split randômico por id
    # ----------------------------
# Divisão treino/teste por tempfranquia
# ----------------------------
    season_to_ids = df.groupby("tempfranquia")["id"].unique().to_dict()

    import random
    random.seed(random_state)

    seasons = list(season_to_ids.keys())
    random.shuffle(seasons)

    n_test = max(1, int(len(seasons) * test_size))
    test_seasons = seasons[:n_test]
    train_seasons = seasons[n_test:]

    train_ids = [i for s in train_seasons for i in season_to_ids[s]]
    test_ids  = [i for s in test_seasons for i in season_to_ids[s]]

    train_groups = [g for g in groups if g["id"] in train_ids]
    test_groups  = [g for g in groups if g["id"] in test_ids]


    # criar scaler a partir de treino
    dataset_train = QueenSequenceDataset(train_groups, scaler=None)
    scaler = dataset_train.scaler
    dataset_test = QueenSequenceDataset(test_groups, scaler=scaler)

    # lidar com imbalance: WeightedRandomSampler por label na dataset_train
    labels_train = np.array([g["label"] for g in train_groups])
    class_counts = Counter(labels_train.tolist())
    # se classe 1 muito rara, dar weight inverso da frequência
    class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}
    sample_weights = np.array([class_weights[int(lbl)] for lbl in labels_train])
    sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

    loader_train = DataLoader(dataset_train, batch_size=batch_size, sampler=sampler, collate_fn=collate_fn)
    loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    input_size = dataset_train[0][0].shape[1]
    model = GRUClassifier(input_size=input_size, hidden_size=hidden_size, bidirectional=True).to(device)

    # Loss com peso de classes (opcional)
    # calcular peso para classe positiva na BCEWithLogitsLoss
    pos_weight = None
    if 1 in class_counts:
        neg = class_counts.get(0, 0)
        pos = class_counts.get(1, 0)
        if pos == 0:
            pos_weight = None
        else:
            pos_weight = torch.tensor([neg / pos], dtype=torch.float32).to(device)

    if pos_weight is not None:
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    else:
        criterion = nn.BCEWithLogitsLoss()

    opt = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

    best_f1 = -1
    best_state = None

    for epoch in range(1, epochs + 1):
        loss = train_epoch(model, loader_train, opt, criterion, device)
        metrics = eval_model(model, loader_test, device)
        print(f"Epoch {epoch:02d} | Loss {loss:.4f} | val f1 {metrics['f1']:.4f} prec {metrics['precision']:.4f} rec {metrics['recall']:.4f} auc {metrics['auc']:.4f}")
        if metrics["f1"] > best_f1:
            best_f1 = metrics["f1"]
            best_state = {k:v.cpu() for k,v in model.state_dict().items()}

    # carregar melhor modelo
    if best_state is not None:
        model.load_state_dict(best_state)
    final_metrics = eval_model(model, loader_test, device)
    print("== Final metrics on test set:", final_metrics)
    return {"model": model, "scaler": scaler, "dataset_train": dataset_train, "dataset_test": dataset_test, "metrics": final_metrics}


In [26]:
cols = ["id", "queen", "ep", "bom", "ruim", "media", "colocacao", "idade", "tempfranquia", "vencedora"]
out = run_pipeline2(df, epochs=10, batch_size=4)

Epoch 01 | Loss 1.4179 | val f1 0.3235 prec 0.1930 rec 1.0000 auc 0.9009
Epoch 02 | Loss 0.8893 | val f1 0.3548 prec 0.2157 rec 1.0000 auc 0.9034
Epoch 03 | Loss 0.8510 | val f1 0.3284 prec 0.1964 rec 1.0000 auc 0.9132
Epoch 04 | Loss 0.7883 | val f1 0.3860 prec 0.2391 rec 1.0000 auc 0.9124
Epoch 05 | Loss 0.7221 | val f1 0.4231 prec 0.2683 rec 1.0000 auc 0.9238
Epoch 06 | Loss 0.6104 | val f1 0.4314 prec 0.2750 rec 1.0000 auc 0.9378
Epoch 07 | Loss 0.5714 | val f1 0.4400 prec 0.2821 rec 1.0000 auc 0.9451
Epoch 08 | Loss 0.4920 | val f1 0.4583 prec 0.2973 rec 1.0000 auc 0.9541
Epoch 09 | Loss 0.6120 | val f1 0.4314 prec 0.2750 rec 1.0000 auc 0.9451
Epoch 10 | Loss 0.4091 | val f1 0.4651 prec 0.3125 rec 0.9091 auc 0.9517
== Final metrics on test set: {'precision': 0.3125, 'recall': 0.9090909090909091, 'f1': 0.4651162790697674, 'auc': 0.9516789516789517}


In [ ]:
results = predict_episode_by_episode(out["model"], out["dataset_test"], device="cpu")
df_table = make_episode_table(results)
print(df_table.to_string(index=False))

df_table.to_excel("AAAAAAAA2.xlsx", index=False)

queen  ep1  ep2  ep3  ep4  ep5  ep6  ep7  ep8  ep9 ep10 ep11 ep12 ep13
  384 0.90 0.94 0.93 0.98 0.98 0.99 0.90                              
  115 0.22 0.06                                                       
  206 0.91 0.91 0.91 0.92 0.96 0.94 0.94 0.96 0.94 0.73 0.39          
  660 0.02 0.02 0.04 0.02                                             
   25 0.84 0.90 0.94 0.92 0.82 0.96 0.95 0.70 0.97 0.96 0.90 0.43     
  233 0.78 0.87 0.86 0.59                                             
  181 0.01 0.01 0.00 0.01                                             
  360 0.02 0.02 0.02                                                  
  434 0.11                                                            
   96 0.02 0.02 0.00 0.01 0.01 0.01                                   
  556 0.09 0.08 0.04                                                  
  510 0.30 0.08                                                       
  439 0.04 0.06 0.02 0.01 0.04 0.01 0.00 0.00 0.01                    
   31 

In [40]:
dim_queen= pd.read_excel('TCC DADOS3.xlsx', sheet_name=0)
dim_queen = dim_queen.rename(columns={"ID": "queen" , "queen":"nome", "temp":"temp", "franquia " : "franquia"})
dim_queen['queen'] = dim_queen['queen'].astype(str)

In [41]:
dim_queen

,queen,nome,idade,colocacao,cidade,estado,temp,franquia
0,1,BeBe Zahara Benet,28,1,Minneapolis,Minnesota,1,1
1,2,Nina Flowers,34,2,Denver,Colorado,1,1
2,3,Rebecca Glasscock,26,3,Fort Lauderdale,Flórida,1,1
3,4,Shannel,29,4,Las Vegas,Nevada,1,1
4,5,Ongina,26,5,Los Angeles,Califórnia,1,1
...,...,...,...,...,...,...,...,...
670,671,Gala Varo,34,8,Morelia,Mexico,19,1
671,672,Soa de Muse,34,9,Saint-Denis,France,19,1
672,673,Eva Le Queen,35,10,Marikina,Philippines,19,1
673,674,Miranda Lebrão,34,11,Rio de Janeiro,Brazil,19,1


In [42]:
results = predict_episode_by_episode(out["model"], out["dataset_test"], device="cpu")
df_table = make_episode_table(results)
print(df_table.to_string(index=False))
df_table = df_table.merge(
    dim_queen[['queen', 'nome']],  # 1. Seleciona as colunas de junção ('queen') e a coluna a adicionar ('nome')
    on='queen',                    # 2. Chave de junção
    how='left'                     # 3. Tipo de junção: Left Join (preserva todas as linhas de df_table)
)

df_table.to_excel("AAAAAAAA2.xlsx", index=False)

queen  ep1  ep2  ep3  ep4  ep5  ep6  ep7  ep8  ep9 ep10 ep11 ep12 ep13
   35 0.95 0.96 0.98 0.97 0.99 0.99 1.00 0.98 0.99 0.97 0.99          
   36 0.86 0.90 0.92 0.96 0.99 0.99 0.99 0.99 0.99 0.98 0.90          
   37 0.03 0.01 0.01 0.01 0.00 0.02 0.08 0.04 0.09 0.71 0.92          
   38 0.02 0.01 0.00 0.01 0.01 0.10 0.07 0.80 0.25 0.10 0.10          
   39 0.01 0.01 0.01 0.00 0.00 0.00                                   
   40 0.02 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00                    
   41 0.72 0.45 0.26 0.27 0.29 0.49 0.14                              
   42 0.01 0.00 0.00 0.00 0.01 0.00 0.01                              
   43 0.03 0.02 0.01 0.00 0.00 0.01                                   
   44 0.34 0.32 0.08 0.01                                             
   45 0.30 0.04 0.01                                                  
   46 0.47 0.05                                                       
   47 0.06                                                            
  249 